In [ ]:
pip install nltk faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 32.7 MB/s eta 0:00:00


In [ ]:
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')


paragraph = """
Elon Reeve Musk is a businessman and entrepreneur known for Tesla and SpaceX.
"""

# Sentence Tokenization
sentences = nltk.sent_tokenize(paragraph)

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

corpus = []

for sentence in sentences:
    text = re.sub('[^a-zA-Z]', ' ', sentence)
    text = text.lower()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    corpus.append(" ".join(words))

print(corpus)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


['elon reeve musk businessman entrepreneur known tesla spacex']


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(corpus)
print(embeddings.shape)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(1, 384)


In [ ]:
import faiss
import numpy as np

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

index.add(np.array(embeddings))

print("Total vectors in DB:", index.ntotal)


Total vectors in DB: 1


In [ ]:
def retrieve_answer(query, top_k=1):
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    return corpus[indices[0][0]]


In [ ]:
query = "Who is Elon Musk?"
answer = retrieve_answer(query)

print("User:", query)
print("Bot:", answer)


User: Who is Elon Musk?
Bot: elon reeve musk businessman entrepreneur known tesla spacex


def add_new_knowledge(new_text):
    sentences = nltk.sent_tokenize(new_text)
    new_corpus = []

    for sentence in sentences:
        text = re.sub('[^a-zA-Z]', ' ', sentence)
        text = text.lower()
        words = text.split()
        words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
        new_corpus.append(" ".join(words))

    new_embeddings = model.encode(new_corpus)
    index.add(np.array(new_embeddings))
    corpus.extend(new_corpus)

    print("Knowledge base updated!")


In [ ]:
new_info = "SpaceX focuses on reusable rockets and Mars missions."

print("Updated DB size:", index.ntotal)



Updated DB size: 1
